In [22]:
import os
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
from snowflake.cortex import Summarize, Complete, ExtractAnswer, Sentiment, Translate
import pandas as pd

In [23]:
load_dotenv()

True

In [24]:
connection_params = {
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_USER_PASSWORD'],
    "role": 'ACCOUNTADMIN',
    #"database": 'SNOWFLAKE_LEARNING_DB',
    "warehouse": 'COMPUTE_WH'
    #"schema": 'PUBLIC',
    }

In [48]:
snowflake_session = Session.builder.configs(connection_params).create()

In [37]:
input_text = '''
    @sleep_and_retry
    @limits(calls=10, period=ONE_MINUTE)
    def check_repo_for_readme(repo_owner, repo_name):

        url = f'https://api.github.com/repos/{repo_owner}/{repo_name}/readme'
        response = requests.get(url, headers=HEADERS, data=PAYLOAD)
        if response.status_code == 404:
            print(f'Repo {repo_name} has no README.md.')
            return False
        elif response.status_code == 200:
            return True
        else:
            if response.status_code != 200:
                print(f'Error with response. Check out status_code {response.status_code}!')
                rate_limit_remaining = int(response.headers.get('X-RateLimit-Remaining'))
                if rate_limit_remaining <= 1:
                    set_sleeper(61)
            return False
'''
input_text = input_text.replace("'", "\\'")

In [38]:
prompt = f'Create a short summary of the given python code. Limit your output to the relevant parts and answer in natural language: "{input_text}"'

In [44]:
print(prompt)

Create a short summary of the given python code. Limit your output to the relevant parts and answer in natural language: "
    @sleep_and_retry
    @limits(calls=10, period=ONE_MINUTE)
    def check_repo_for_readme(repo_owner, repo_name):

        url = f\'https://api.github.com/repos/{repo_owner}/{repo_name}/readme\'
        response = requests.get(url, headers=HEADERS, data=PAYLOAD)
        if response.status_code == 404:
            print(f\'Repo {repo_name} has no README.md.\')
            return False
        elif response.status_code == 200:
            return True
        else:
            if response.status_code != 200:
                print(f\'Error with response. Check out status_code {response.status_code}!\')
                rate_limit_remaining = int(response.headers.get(\'X-RateLimit-Remaining\'))
                if rate_limit_remaining <= 1:
                    set_sleeper(61)
            return False
"


In [ ]:
df = snowflake_session.sql(f"select snowflake.cortex.complete('llama3-8b', '{prompt}') as response").to_pandas()
# snowflake-llama-3.3-70b

In [53]:
df_testi = df.iloc[0]['RESPONSE']

In [56]:
print(df_testi)

This Python function, `check_repo_for_readme`, checks if a GitHub repository has a README.md file. It makes a GET request to the repository's API endpoint and checks the response status code. If the status code is 404, it means the repository has no README.md file and the function returns `False`. If the status code is 200, it means the repository has a README.md file and the function returns `True`. If the status code is anything else, it prints an error message and returns `False`. The function also handles rate limiting by the GitHub API, pausing for 61 seconds if the remaining rate limit is 1 or less.


This Python function, `check_repo_for_readme`, checks if a GitHub repository has a README.md file. 
It makes a GET request to the repository's API endpoint and checks the response status code. 
If the status code is 404, it means the repository has no README.md file and the function returns `False`. 
If the status code is 200, it means the repository has a README.md file and the function returns `True`. 
If the status code is anything else, it prints an error message and returns `False`. 
The function also handles rate limiting by the GitHub API, pausing for 61 seconds if the remaining rate limit is 1 or less.

In [58]:
snowflake_session.close()

In [50]:
print(prompt)

Create a short summary of the given python code. Limit your output to the relevant parts and answer in natural language: "
    @sleep_and_retry
    @limits(calls=10, period=ONE_MINUTE)
    def check_repo_for_readme(repo_owner, repo_name):

        url = f\'https://api.github.com/repos/{repo_owner}/{repo_name}/readme\'
        response = requests.get(url, headers=HEADERS, data=PAYLOAD)
        if response.status_code == 404:
            print(f\'Repo {repo_name} has no README.md.\')
            return False
        elif response.status_code == 200:
            return True
        else:
            if response.status_code != 200:
                print(f\'Error with response. Check out status_code {response.status_code}!\')
                rate_limit_remaining = int(response.headers.get(\'X-RateLimit-Remaining\'))
                if rate_limit_remaining <= 1:
                    set_sleeper(61)
            return False
"


In [ ]:
# count tokens with snowflake
from snowflake.snowpark.context import get_active_session

# Get current Snowflake session
session = get_active_session()

# Define model and input text
model_name = "llama3-8b"

# Execute token count function
result = session.sql(f"SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS('{model_name}', '{prompt}') AS token_count").collect()

print(f"Token count: {result[0]['TOKEN_COUNT']}") # 204 tokens (snowflake)


Token count: 203


In [ ]:
from transformers import AutoTokenizer

# Load tokenizer for a specific model
tokenizer = AutoTokenizer.from_pretrained("facebook/llama3-8b")

tokens = tokenizer.encode(prompt)

print(f"Token count: {len(tokens)}")


ModuleNotFoundError: No module named 'transformers'